In [7]:
import torch as th
import numpy as np
from functions_ngates_v2 import calc_variance_pure, calc_variance_ng_cnot_batched, _kron_batched
from functions_ngates_v1 import calc_variance_ng_crx_batched

In [8]:
def _rx(phi, device='cpu'):
    batch, s1, s2 = phi.shape[0], phi.shape[1], phi.shape[2]
    rx = th.zeros(batch, s1, s2, 2, 2, dtype=th.complex128, device=device)
    rx[:,:,:,0,0] = th.cos(phi/2)
    rx[:,:,:,1,1] = th.cos(phi/2)
    rx[:,:,:,0,1] = -1j*th.sin(phi/2)
    rx[:,:,:,1,0] = -1j*th.sin(phi/2)

    return rx

def _ry(phi, device='cpu'):
    batch, s1, s2 = phi.shape[0], phi.shape[1], phi.shape[2]
    ry = th.zeros(batch, s1, s2, 2, 2, dtype=th.complex128, device=device)
    ry[:,:,:,0,0] = th.cos(phi/2)
    ry[:,:,:,1,1] = th.cos(phi/2)
    ry[:,:,:,0,1] = -th.sin(phi/2)
    ry[:,:,:,1,0] = th.sin(phi/2)

    return ry

def _Rxx(phi, device='cpu'):
    batch, s1, s2 = phi.shape[0], phi.shape[1], phi.shape[2]
    Rxx = th.zeros(batch, s1, s2, 4, 4, dtype=th.complex128, device=device)
    Rxx[:,:,:,0,0] = th.cos(phi/2)
    Rxx[:,:,:,1,1] = th.cos(phi/2)
    Rxx[:,:,:,2,2] = th.cos(phi/2)
    Rxx[:,:,:,3,3] = th.cos(phi/2)
    Rxx[:,:,:,0,3] = -1j*th.sin(phi/2)
    Rxx[:,:,:,1,2] = -1j*th.sin(phi/2)
    Rxx[:,:,:,2,1] = -1j*th.sin(phi/2)
    Rxx[:,:,:,3,0] = -1j*th.sin(phi/2)
    
    return Rxx

def build_cnot_layer_operator_batch(N, L, phi, device='cpu'):
    """
    Constructs a batch of CNOT layer operators.
    phi_batch has shape (batch_size, N-1).
    Returns a tensor of shape (batch_size, 2**N, 2**N).
    """
    batch_size = phi.shape[0]
    
    # Pre-calculate identities to reuse
    identities = [th.eye(2**i, dtype=th.complex128, device=device).repeat(batch_size, L, 1, 1) for i in range(N + 1)]

    cnot = th.exp(-1j * phi.unsqueeze(-1).unsqueeze(-1)) *_kron_batched(_ry(-2*phi) @ _rx(-2*phi), _rx(-2*phi)) @ _Rxx(2*phi) @ _kron_batched(_ry(2*phi),th.eye(2, dtype=th.complex128, device=device))

    full_ops = _kron_batched(identities[0], _kron_batched(cnot[:, 0], identities[N-2]))
    
    for j in range(1, N-1):
        Id1 = identities[j]
        Id2 = identities[N-j-2]
        full_ops = full_ops @ _kron_batched(Id1, _kron_batched(cnot[:, j], Id2))

    return cnot

In [57]:
L = 1
N = 2

batch_size = 1

phi = th.randn(size=(batch_size, N-1, L), dtype=th.float64) * np.pi/4

cnot = th.zeros((batch_size, N-1, L, 4, 4), dtype=th.complex128,)
cnot[:,:,:,0,0] = 1
cnot[:,:,:,1,1] = 1
cnot[:,:,:,2,2] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,3] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,2,3] = (1 - th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,2] = (1 - th.exp(1j * 4 * phi))/2

cnot

tensor([[[[[1.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
           [0.0000+0.0000j, 1.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
           [0.0000+0.0000j, 0.0000+0.0000j, 0.0169+0.1290j, 0.9831-0.1290j],
           [0.0000+0.0000j, 0.0000+0.0000j, 0.9831-0.1290j, 0.0169+0.1290j]]]]],
       dtype=torch.complex128)

In [60]:
cnot[:,0,0] @ cnot[:,0,0].conj().transpose(-1,-2)

tensor([[[ 1.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
         [ 0.0000e+00+0.j,  1.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
         [ 0.0000e+00+0.j,  0.0000e+00+0.j,  1.0000e+00+0.j, -4.8572e-17+0.j],
         [ 0.0000e+00+0.j,  0.0000e+00+0.j, -4.8572e-17+0.j,  1.0000e+00+0.j]]],
       dtype=torch.complex128)

In [22]:
cnot = th.zeros((batch_size, N-1, L, 4, 4), dtype=th.complex128,)
cnot[:,:,:,0,0] = 1
cnot[:,:,:,1,1] = 1
cnot[:,:,:,2,2] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,3] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,2,3] = (1 - th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,2] = (1 - th.exp(1j * 4 * phi))/2
np.round(cnot.numpy())

array([[[[[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.-0.j],
          [0.+0.j, 0.+0.j, 0.-0.j, 1.+0.j]]]],



       [[[[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j],
          [0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j]]]]])

In [4]:
calc_variance_ng_cnot_batched(4,10,n_sim=1000, n_sim_noise=500).shape

torch.Size([1000, 500])

In [6]:
calc_variance_ng_crx_batched(4,10,np.pi/2,n_sim=1000,n_sim_noise=500).shape

torch.Size([1000, 500])

In [ ]:
import torch.nn as nn

class circuit(nn.Module):
    def __init__(self, phi_init=phi):
        super().__init__()
        self.phi = nn.Parameter(torch.tensor(phi_init, dtype=torch.float64))

    def forward(self):
        # Example: apply Rx[0] and Ry[0] using the optimizable phi
        rx_mat = Rx[0](self.phi)
        ry_mat = Ry[0](self.phi)
        return rx_mat @ ry_mat

# Instantiate the module
opt_module = MyOptimModule()